In [29]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import glob
import numbers
from datetime import datetime
from selenium import webdriver
from tableauscraper import TableauScraper as TS
from bs4 import BeautifulSoup

## VAKSIN

In [30]:
import time

In [31]:
url ='https://public.tableau.com/views/DashboardVaksinKemkes/TotalVaksinasiDosis1?:embed=yes&:showVizHome=no'
driver = webdriver.Chrome("C:/Program Files (x86)/chromedriver.exe")
driver.get(url)

time.sleep(4)
a = driver.find_elements_by_xpath("//div[contains(@class, 'tab-textRegion-content tab-selectable')]")
b = a[0].find_elements_by_xpath(".//span")

dosis1 = b[3].text.split()[0].replace(',','.')
persen_dosis1 = b[4].text

url2 = 'https://public.tableau.com/views/DashboardVaksinKemkes/TotalVaksinasiDosis2?:embed=yes&:showVizHome=no'
driver.get(url2)

time.sleep(4)
a = driver.find_elements_by_xpath("//div[contains(@class, 'tab-textRegion-content tab-selectable')]")
b = a[0].find_elements_by_xpath(".//span")

dosis2 = b[4].text.split()[0].replace(',','.')
persen_dosis2 = b[7].text

print(dosis1)
print(dosis2)
print(persen_dosis1)
print(persen_dosis2)

driver.quit()

131.675.335
85.761.511
63.22%
41.18%


## NON VAKSIN & RATA2 LAJU VAKSIN

In [32]:
driver = webdriver.Chrome("C:/Program Files (x86)/chromedriver.exe")
driver.get("https://www.worldometers.info/coronavirus/#countries")
data = pd.read_html(driver.page_source)[0]

In [33]:
def konversi_integer(array):
    wadah1 = []
    for i in array:
        try:
            pra = i.replace('+','')
            ent = int(i.replace(',',''))
        except:
            ent = i

        wadah1 = wadah1 + [ent]
    
    return wadah1

data['NewCases'] = konversi_integer(data['NewCases'].values)
data['ActiveCases'] = konversi_integer(data['ActiveCases'].values)
data['NewDeaths'] = konversi_integer(data['NewDeaths'].values)

In [35]:
def coma_rep(df):
    
    df_wadah = pd.DataFrame(columns = df.columns)
    
    for i in range(len(df)):
        identity = list(df.iloc[i,:2].values)
        row = list(df.iloc[i, 2:].values)
        row_int = [int(str(i).replace(',','')) for i in row]
        gabungan_row = identity + row_int
        df_wadah.loc[i,:] = gabungan_row
    
    df_wadah_final = df_wadah.reset_index(drop=True)
    
    return df_wadah_final

scope = [
"https://spreadsheets.google.com/feeds",
'https://www.googleapis.com/auth/spreadsheets',
"https://www.googleapis.com/auth/drive.file",
"https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("premise-530f3762fbef.json", scope)
client = gspread.authorize(creds)
sheet = client.open("COVID-19")

In [36]:
#laju vaksin 7 hari terakhir

vaksinasi = pd.DataFrame(sheet.worksheet("Vaksinasi Nasional").get_all_records())
df_vaksinasi = vaksinasi[vaksinasi['Tanggal'].notna()]
df_vaksinasi.replace({"": np.nan}, inplace=True)
vaksinasi_final = df_vaksinasi[df_vaksinasi['Tanggal'].notna()].iloc[:,:9]
rata_dosis1 = vaksinasi_final['Harian Vaksin 1'].iloc[-7:].mean().astype(int)
rata_dosis2 = vaksinasi_final['Harian Vaksin 2'].iloc[-7:].mean().astype(int)
rata_dosis1 = f'{rata_dosis1:,}'.replace(',','.')
rata_dosis2 = f'{rata_dosis2:,}'.replace(',','.')
penambahan_dosis1 = vaksinasi_final['Harian Vaksin 1'].iloc[-1]
penambahan_dosis2 = vaksinasi_final['Harian Vaksin 2'].iloc[-1]
penambahan_dosis1 = f'{penambahan_dosis1:,}'.replace(',','.')
penambahan_dosis2 = f'{penambahan_dosis2:,}'.replace(',','.')

In [55]:
matching = [s for s in kolom_kasusbaru if "2021" in s]
# last_index = np.where(kolom_kasusbaru == matching[-1])[0][0]
# kasusbaru_final = tabel_kasusbaru_prov.iloc[:35, :last_index+1]
matching[-1]

'2021-11-16'

In [44]:
# rata_dosis1 = int(np.round(np.nanmean(hihi.sum(axis=0)[-7:]), 0))
# rata_dosis2 = int(np.round(np.nanmean(hihi2.sum(axis=0)[-7:]), 0))

def coma_rep(df):
    
    df_wadah = pd.DataFrame(columns = df.columns)
    
    for i in range(len(df)):
        identity = list(df.iloc[i,:2].values)
        row = list(df.iloc[i, 2:].values)
        row_int = [int(str(i).replace(',','')) for i in row]
        gabungan_row = identity + row_int
        df_wadah.loc[i,:] = gabungan_row
    
    df_wadah_final = df_wadah.reset_index(drop=True)
    
    return df_wadah_final


# SPESIMEN ==============k=========================================================
spesimen = pd.DataFrame(sheet.worksheet("Spesimen").get_all_records())
df_testing = spesimen[spesimen['Tanggal'].notna()]
df_testing.replace({"": np.nan}, inplace=True)
testing_final = df_testing[df_testing['Tanggal'].notna()].iloc[:,:9]
testing_final['persentase'] = np.round((testing_final['harian kasus'] / testing_final['harian orang'])*100, 2)

# AKTIF =======================================================================
tabel_aktif_prov = pd.DataFrame(sheet.worksheet("Active Case (Prov)").get_all_records())
kolom_aktif = tabel_aktif_prov.columns

matching = [s for s in kolom_aktif if "2021" in s]
last_index = np.where(kolom_aktif == matching[-1])[0][0]
tabel_aktif_prov = tabel_aktif_prov.iloc[:35, :last_index+1]
aktif_final = coma_rep(tabel_aktif_prov)

# AKUMULATIF =======================================================================
tabel_akum_prov = pd.DataFrame(sheet.worksheet("Confirm (Prov)").get_all_records())
kolom_akum = tabel_akum_prov.columns

matching = [s for s in kolom_akum if "2021" in s]
last_index = np.where(kolom_akum == matching[-1])[0][0]
akum_final = tabel_akum_prov.iloc[:35, :last_index+1]

# SEMBUH =======================================================================
tabel_sembuh_prov = pd.DataFrame(sheet.worksheet("Recovered (Prov)").get_all_records())
kolom_sembuh = tabel_sembuh_prov.columns

matching = [s for s in kolom_sembuh if "2021" in s]
last_index = np.where(kolom_sembuh == matching[-1])[0][0]
sembuh_final = tabel_sembuh_prov.iloc[:35, :last_index+1]

# MENINGGAL =======================================================================
tabel_mati_prov = pd.DataFrame(sheet.worksheet("Death (Prov)").get_all_records())
kolom_mati = tabel_mati_prov.columns

matching = [s for s in kolom_mati if "2021" in s]
last_index = np.where(kolom_mati == matching[-1])[0][0]
mati_final = tabel_mati_prov.iloc[:35, :last_index+1]

# KASUS BARU =======================================================================
tabel_kasusbaru_prov = pd.DataFrame(sheet.worksheet("New Case (Prov)").get_all_records())
kolom_kasusbaru = tabel_kasusbaru_prov.columns

matching = [s for s in kolom_kasusbaru if "2021" in s]
last_index = np.where(kolom_kasusbaru == matching[-1])[0][0]
kasusbaru_final = tabel_kasusbaru_prov.iloc[:35, :last_index+1]

# PREDIKSI =======================================================================
tabel_prediksi = pd.DataFrame(sheet.worksheet("Prediksi").get_all_records())
tabel_prediksi.drop([2,3], inplace=True)
kolom_prediksi = tabel_prediksi.columns

matching = [s for s in kolom_prediksi if "2021" in s]
last_index = np.where(kolom_prediksi == matching[-1])[0][0]
prediksi_final = tabel_prediksi.iloc[:, :last_index+1]

prediksi_tudey = tabel_prediksi[akum_final.columns[-1]].values[:-1]
prediksi_tudey = [x for x in prediksi_tudey if isinstance(x, numbers.Number)]
dua_skenario = ['Aman','Rentan']

prediksi_terdekat = min(prediksi_tudey, key=lambda x:abs(x-kasusbaru_final.iloc[-1,-1]))
indeks_skenario = np.where(np.array(prediksi_tudey) == prediksi_terdekat)[0][0]
skenario = dua_skenario[indeks_skenario]
akurasi = np.round(min([prediksi_terdekat, kasusbaru_final.iloc[-1,-1]]) / max([prediksi_terdekat, kasusbaru_final.iloc[-1,-1]])*100, 2)

if min([prediksi_terdekat, akum_final.iloc[-1,-1]]) == akum_final.iloc[-1,-1]:
    atasbawah = 'dibawah'
else:
    atasbawah = 'diatas'
    
angka_prediksi = f"{prediksi_terdekat:,}".replace(',','.')

bulan_indo = ["Januari","Februari","Maret","April","Mei","Juni","Juli","Agustus","September","Oktober","November","Desember"]

#################### PARAMETRIK ###################
hari_ini = str(datetime.today().day) + ' ' + bulan_indo[datetime.today().month-1] + ' ' + str(datetime.today().year)
akumulatif_nasional = f'{akum_final.iloc[-1,-1]:,}'.replace(',','.')
sembuh_nasional = f'{sembuh_final.iloc[-1,-1]:,}'.replace(',','.')
tambah_sembuh = sembuh_final.iloc[-1,-1] - sembuh_final.iloc[-1,-2]
sembuh_baru = f'{tambah_sembuh:,}'.replace(',','.')
kematian_nasional = f'{mati_final.iloc[-1,-1]:,}'.replace(',','.')
tambah_kematian = mati_final.iloc[-1,-1] - mati_final.iloc[-1,-2]
mati_baru = f'{tambah_kematian:,}'.replace(',','.')
kasusbaru_nasional = akum_final.iloc[-1,-1] - akum_final.iloc[-1,-2]
kasusbaru_nasional = f'{kasusbaru_nasional:,}'.replace(',','.')
aktif_nasional_pra = akum_final.iloc[-1,-2] - (sembuh_final.iloc[-1,-2] + mati_final.iloc[-1,-2])
aktif_nasional_angka = akum_final.iloc[-1,-1] - (sembuh_final.iloc[-1,-1] + mati_final.iloc[-1,-1])
aktif_nasional = f'{aktif_nasional_angka:,}'.replace(',','.')
tambah_aktif = aktif_nasional_angka - aktif_nasional_pra
aktif_baru = f'{tambah_aktif:,}'.replace(',','.')
rasio_aktif = round((aktif_final.iloc[-1,-2]/akum_final.iloc[-1,-2])*100,2)
aktif_5provinsi = aktif_final[:-1].sort_values(by=aktif_final.columns[-1], ascending=False).Provinsi.values[:5]
angka_5provinsi = aktif_final[:-1].sort_values(by=aktif_final.columns[-1], ascending=False)[aktif_final.columns[-1]][:5].values
angka_5provinsi = [f'{i:,}'.replace(',','.') for i in angka_5provinsi]

orang_ditest = int(testing_final['harian orang'].values[-1])
orang_tudey = f'{orang_ditest:,}'.replace(',','.')

spesimen_ditest = int(testing_final['Spesimen harian'].values[-1])
spesimen_tudey = f'{spesimen_ditest:,}'.replace(',','.')

posrate_orang = testing_final['persentase'].values[-1]
angka_orang = int(float(posrate_orang))

data.loc[np.where(data['Country,Other'] == 'Indonesia')[0], 'NewCases'] = int(kasusbaru_final.iloc[-1,-1])
data.loc[np.where(data['Country,Other'] == 'Indonesia')[0], 'ActiveCases'] = int(aktif_final.iloc[-1,-1])
data.loc[np.where(data['Country,Other'] == 'Indonesia')[0], 'NewDeaths'] = int(tambah_kematian)

kasbar = data.sort_values(by='NewCases', ascending=False).reset_index(drop=True).drop([0,1]).reset_index(drop=True)
kasmit = data.sort_values(by='NewDeaths', ascending=False).reset_index(drop=True).drop([0,1]).reset_index(drop=True)
posisi_baru = np.where(kasbar['Country,Other'] == 'Indonesia')[0][0] + 1
posisi_kematian = np.where(kasmit['Country,Other'] == 'Indonesia')[0][0] + 1

wew = [kasusbaru_final.iloc[15,-1],
 kasusbaru_final.iloc[8,-1],
 kasusbaru_final.iloc[3,-1],
 kasusbaru_final.iloc[6,-1],
 kasusbaru_final.iloc[20,-1],
 kasusbaru_final.iloc[13,-1],
 kasusbaru_final.iloc[14,-1],
 kasusbaru_final.iloc[25,-1],
 kasusbaru_final.iloc[0,-1]
]

wew.sort(reverse=True)
wew2 = wew.copy()
wew = [kasusbaru_final.iloc[15,-1],
 kasusbaru_final.iloc[8,-1],
 kasusbaru_final.iloc[3,-1],
 kasusbaru_final.iloc[6,-1],
 kasusbaru_final.iloc[20,-1],
 kasusbaru_final.iloc[13,-1],
 kasusbaru_final.iloc[14,-1],
 kasusbaru_final.iloc[25,-1],
 kasusbaru_final.iloc[0,-1]
]

indexing = [np.where(np.array(wew) == wew2[i])[0][0] for i in range(len(wew2))]
wedew = [f'*- Jawa Barat ({kasusbaru_final.iloc[15,-1]} Kasus Baru)*, penularan akibat mutasi varian delta semakin banyak',
f'*- Jawa Timur ({kasusbaru_final.iloc[8,-1]} Kasus Baru)*, penularan dari transmisi lokal',
f'*- Jawa Tengah ({kasusbaru_final.iloc[3,-1]} Kasus Baru)*, berasal dari penularan yang cepat akibat mutasi varian delta yang sangat signifikan dan aktivitas masyarakat lokal.',
f'*- Sumatera Utara ({kasusbaru_final.iloc[6,-1]} Kasus Baru)*, didominasi akibat transmisi aktif dari orang tanpa gejala yang menularkan virus COVID-19 ke tenaga kesehatan di Puskesmas.',
f'*- DKI Jakarta ({kasusbaru_final.iloc[20,-1]} Kasus Baru)*, berasal dari klaster rumah tangga atau keluarga dan komunitas serta kluster perkantoran dengan penyebaran akibat mutasi varian delta.',
f'*- Sulawesi Selatan ({kasusbaru_final.iloc[13,-1]} Kasus Baru)*, disebabkan karena klaster keluarga dan transmisi lokal.',
f'*- Bali ({kasusbaru_final.iloc[14,-1]} Kasus Baru)*, didominasi oleh kasus transmisi lokal yang terjadi dan klaster pelaku perjalanan dalam negeri.',
f'*- Kalimantan Timur ({kasusbaru_final.iloc[25,-1]} Kasus Baru)*, varian delta diduga semakin masif dampaknya dengan peningkatan kasus',
f'*- Aceh ({kasusbaru_final.iloc[0,-1]} Kasus Baru)*, penularan dari transmisi lokal']

hasil_index = [wedew[i] for i in indexing]

persen_epis = np.round((np.sum(wew) / kasusbaru_final.iloc[-1,-1])*100,2)
# rata_dosis1 = f'{rata_dosis1:,}'.replace(',','.')

WA_MODEL = f"""Kepada Yth. : 

*1. Rajawali*
*2. Elang*

Dari : *Kenari-02*

Perihal : *Laporan Perkembangan COVID-19 & Rekor Tertinggi di Dunia* 

Tanggal: *{hari_ini}*

Bersama ini dengan hormat dilaporkan Perkembangan Kasus COVID-19, sebagai berikut :

*Kasus Positif* : {akumulatif_nasional} kasus *(+{kasusbaru_nasional})*
*Sembuh* : {sembuh_nasional} kasus *(+{sembuh_baru})*
*Meninggal* : {kematian_nasional} kasus *(+{mati_baru})*
*Kasus Aktif* : {aktif_nasional} kasus *({aktif_baru})* berasal dari jumlah kasus sembuh dan meninggal yang lebih tinggi dibandingkan jumlah kasus baru
*Suspek* : _____ kasus
*Orang di Tes* : {orang_tudey} orang
*_Positivity Rate_* : {posrate_orang}%

Dengan kondisi tersebut (update per pukul 17.00), *Indonesia menjadi negara dengan:*
*1. Kasus baru harian peringkat {posisi_baru} dunia.* Lima negara dengan kasus baru tertinggi hari ini adalah {kasbar['Country,Other'].values[0]}, {kasbar['Country,Other'].values[1]}, {kasbar['Country,Other'].values[2]}, {kasbar['Country,Other'].values[3]}, dan {kasbar['Country,Other'].values[4]}
*2. Kasus kematian baru peringkat {posisi_kematian} dunia.* Lima negara dengan kasus kematian baru tertinggi hari ini adalah {kasbar['Country,Other'].values[0]}, {kasmit['Country,Other'].values[1]}, {kasmit['Country,Other'].values[2]}, {kasmit['Country,Other'].values[3]}, dan {kasmit['Country,Other'].values[4]}

Pemerintah menetapkan total target vaksinasi sebanyak 208.265.720 orang pada tenaga kesehatan, petugas publik, lansia, masyarakat rentan dan masyarakat umum, usia 12 - 17 tahun. Adapun jumlah orang yang telah menerima vaksin COVID-19 di Indonesia per {hari_ini} pukul 12.00 WIB, sebagai berikut :
*- Dosis pertama: {dosis1} (+{penambahan_dosis1})* orang, artinya {persen_dosis1} dari total target telah divaksin dosis pertama
*- Dosis kedua: {dosis2} (+{penambahan_dosis2})* orang, artinya {persen_dosis2} dari total target telah divaksin dosis kedua

Rata-rata harian jumlah suntikan dalam seminggu terakhir:
*- Dosis pertama: {rata_dosis1}* orang
*- Dosis kedua: {rata_dosis2}* orang
	
Munculnya kasus baru dengan jumlah yang tinggi ( *{persen_epis}%* dari total jumlah kasus baru ), masih berasal dari Episentrum (9 dari 34 Provinsi). Adapun rinciannya adalah sebagai berikut :
{hasil_index[0]}
{hasil_index[1]}
{hasil_index[2]}
{hasil_index[3]}
{hasil_index[4]}
{hasil_index[5]}
{hasil_index[6]}
{hasil_index[7]}
{hasil_index[8]}

*DMMP*"""

# write data in a file.
file1 = open(f"{hari_ini} PERKEMBANGAN COVID19.txt","w")
  
# \n is placed to indicate EOL (End of Line)
file1.write(WA_MODEL)
file1.close() #to change file access modes

In [45]:
WA_MODEL = f"""Kepada Yth. : 

*Cenderawasih*

Dari : *Kenari-02*

Perihal : *Laporan Perkembangan COVID-19 & Rekor Tertinggi di Dunia* 

Tanggal: *{hari_ini}*

Bersama ini dengan hormat dilaporkan Perkembangan Kasus COVID-19, sebagai berikut :

*Kasus Positif* : {akumulatif_nasional} kasus *(+{kasusbaru_nasional})*
*Sembuh* : {sembuh_nasional} kasus *(+{sembuh_baru})*
*Meninggal* : {kematian_nasional} kasus *(+{mati_baru})*
*Kasus Aktif* : {aktif_nasional} kasus *({aktif_baru})* berasal dari jumlah kasus sembuh dan meninggal yang lebih tinggi dibandingkan jumlah kasus baru
*Suspek* : _____ kasus
*Orang di Tes* : {orang_tudey} orang
*_Positivity Rate_* : {posrate_orang}%

Dengan kondisi tersebut (update per pukul 17.00), *Indonesia menjadi negara dengan:*
*1. Kasus baru harian peringkat {posisi_baru} dunia.* Lima negara dengan kasus baru tertinggi hari ini adalah {kasbar['Country,Other'].values[0]}, {kasbar['Country,Other'].values[1]}, {kasbar['Country,Other'].values[2]}, {kasbar['Country,Other'].values[3]}, dan {kasbar['Country,Other'].values[4]}
*2. Kasus kematian baru peringkat {posisi_kematian} dunia.* Lima negara dengan kasus kematian baru tertinggi hari ini adalah {kasbar['Country,Other'].values[0]}, {kasmit['Country,Other'].values[1]}, {kasmit['Country,Other'].values[2]}, {kasmit['Country,Other'].values[3]}, dan {kasmit['Country,Other'].values[4]}

Pemerintah menetapkan total target vaksinasi sebanyak 208.265.720 orang pada tenaga kesehatan, petugas publik, lansia, masyarakat rentan dan masyarakat umum, usia 12 - 17 tahun. Adapun jumlah orang yang telah menerima vaksin COVID-19 di Indonesia per {hari_ini} pukul 12.00 WIB, sebagai berikut :
*- Dosis pertama: {dosis1} (+{penambahan_dosis1})* orang, artinya {persen_dosis1} dari total target telah divaksin dosis pertama
*- Dosis kedua: {dosis2} (+{penambahan_dosis2})* orang, artinya {persen_dosis2} dari total target telah divaksin dosis kedua

Rata-rata harian jumlah suntikan dalam seminggu terakhir:
*- Dosis pertama: {rata_dosis1}* orang
*- Dosis kedua: {rata_dosis2}* orang
	
Munculnya kasus baru dengan jumlah yang tinggi ( *{persen_epis}%* dari total jumlah kasus baru ), masih berasal dari Episentrum (9 dari 34 Provinsi). Adapun rinciannya adalah sebagai berikut :
{hasil_index[0]}
{hasil_index[1]}
{hasil_index[2]}
{hasil_index[3]}
{hasil_index[4]}
{hasil_index[5]}
{hasil_index[6]}
{hasil_index[7]}
{hasil_index[8]}

*DMMP*"""

# write data in a file.
file1 = open(f"{hari_ini} PERKEMBANGAN COVID19 CENDERAWASIH.txt","w")
  
# \n is placed to indicate EOL (End of Line)
file1.write(WA_MODEL)
file1.close() #to change file access modes

In [46]:
driver.quit()